## 1小时入门Spark之MLlib

MLlib是Spark的机器学习库，包括以下主要功能。

* 实用工具：线性代数，统计，数据处理等工具
* 特征工程：特征提取，特征转换，特征选择
* 常用算法：分类，回归，聚类，协同过滤，降维
* 模型优化：模型评估，参数优化。

MLlib库包括两个不同的部分

spark.mllib 包含基于rdd的机器学习算法API，目前不再更新，在3.0版本后将会丢弃，不建议使用。

spark.ml 包含基于DataFrame的机器学习算法API，可以用来构建机器学习工作流Pipeline，推荐使用。

In [1]:
val rdd = sc.parallelize(1 to 100)
rdd.count

rdd = ParallelCollectionRDD[0] at parallelize at <console>:27


100

In [1]:
%AddDeps org.apache.spark spark_mllib_2.11 2.3.1  

Marking org.apache.spark:spark_mllib_2.11:2.3.1 for download
-> Failed to resolve org.apache.spark:spark_mllib_2.11:2.3.1
    -> not found: /var/folders/bs/pw656yts35qb1dpr1myl3wd80000gn/T/toree-tmp-dir1813761629305680892/toree_add_deps/cache/org.apache.spark/spark_mllib_2.11/ivy-2.3.1.xml
    -> not found: https://repo1.maven.org/maven2/org/apache/spark/spark_mllib_2.11/2.3.1/spark_mllib_2.11-2.3.1.pom
Obtained 0 files


### 一，MLlib基本概念

DataFrame: MLlib中数据的存储形式，其列可以存储特征向量，标签，以及原始的文本，图像。

Transformer：转换器。具有transform方法。通过附加一个或多个列将一个DataFrame转换成另外一个DataFrame。

Estimator：估计器。具有fit方法。它接受一个DataFrame数据作为输入后经过训练，产生一个转换器Transformer。

Pipeline：流水线。具有setStages方法。顺序将多个Transformer和1个Estimator串联起来，得到一个流水线模型。


### 二，Pipeline流水线范例

任务描述：用逻辑回归模型预测句子中是否包括”spark“这个单词。



In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.ml.{Pipeline,PipelineModel}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row


In [3]:
val spark = SparkSession.builder()
   .master("local[4]").appName("ml")
   .getOrCreate()

import spark.implicits._

spark = org.apache.spark.sql.SparkSession@8a0157


org.apache.spark.sql.SparkSession@8a0157

**1，准备数据**

In [4]:
val dftrain = Seq((0L,"a b c d e spark",1.0),
                (1L,"a c f",0.0),
                (2L,"spark hello world",1.0),
                (3L,"hadoop mapreduce",0.0),
                (4L,"I love spark", 1.0),
                (5L,"big data",0.0)).toDF("id","text","label")
dftrain.show

+---+-----------------+-----+
| id|             text|label|
+---+-----------------+-----+
|  0|  a b c d e spark|  1.0|
|  1|            a c f|  0.0|
|  2|spark hello world|  1.0|
|  3| hadoop mapreduce|  0.0|
|  4|     I love spark|  1.0|
|  5|         big data|  0.0|
+---+-----------------+-----+



dftrain = [id: bigint, text: string ... 1 more field]


[id: bigint, text: string ... 1 more field]

**2，构建模型**

In [5]:
val tokenizer = new Tokenizer().setInputCol("text").setOutputCol("words")
println(tokenizer.getClass)

val hashingTF = new HashingTF().setNumFeatures(100)
   .setInputCol(tokenizer.getOutputCol)
   .setOutputCol("features")
println(hashingTF.getClass)

val lr = new LogisticRegression().setLabelCol("label")
//println(lr.explainParams)
lr.setFeaturesCol("features").setMaxIter(10).setRegParam(0.01)
println(lr.getClass)

val pipe = new Pipeline().setStages(Array(tokenizer,hashingTF,lr))
println(pipe.getClass)

class org.apache.spark.ml.feature.Tokenizer
class org.apache.spark.ml.feature.HashingTF
class org.apache.spark.ml.classification.LogisticRegression
class org.apache.spark.ml.Pipeline


tokenizer = tok_fbbc18a06d49
hashingTF = hashingTF_36b0bc22a76f
lr = logreg_93cb4fe68c3e
pipe = pipeline_3ce68f9bab3a


pipeline_3ce68f9bab3a

**3，训练模型**

In [6]:
val model = pipe.fit(dftrain)
print(model.getClass)

class org.apache.spark.ml.PipelineModel

model = pipeline_3ce68f9bab3a


pipeline_3ce68f9bab3a

**4，使用模型**

In [7]:
val dftest = Seq((7L,"spark job",1.0),(9L,"hello world",0.0),
                 (10L,"a b c d e",0.0),(11L,"you can you up",0.0),
                (12L,"spark is easy to use.",1.0)).toDF("id","text","label")
dftest.show

val dfresult = model.transform(dftest)

dfresult.selectExpr("text","features","probability","prediction").show

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  7|           spark job|  1.0|
|  9|         hello world|  0.0|
| 10|           a b c d e|  0.0|
| 11|      you can you up|  0.0|
| 12|spark is easy to ...|  1.0|
+---+--------------------+-----+

+--------------------+--------------------+--------------------+----------+
|                text|            features|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|           spark job|(100,[5,70],[1.0,...|[0.35046042897666...|       1.0|
|         hello world|(100,[48,50],[1.0...|[0.33560921515513...|       1.0|
|           a b c d e|(100,[22,61,70,78...|[0.19082246657270...|       1.0|
|      you can you up|(100,[25,28,33],[...|[0.81519423235140...|       0.0|
|spark is easy to ...|(100,[5,21,60,81,...|[0.47768327161194...|       1.0|
+--------------------+--------------------+--------------------+----------+



dftest = [id: bigint, text: string ... 1 more field]
dfresult = [id: bigint, text: string ... 6 more fields]


[id: bigint, text: string ... 6 more fields]

**5，评估模型**

In [8]:
dfresult.printSchema

root
 |-- id: long (nullable = false)
 |-- text: string (nullable = true)
 |-- label: double (nullable = false)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [9]:
val evaluator = new MulticlassClassificationEvaluator().setMetricName("f1")
    .setPredictionCol("prediction").setLabelCol("label")

println(evaluator.explainParams())

evaluator.evaluate(dfresult)

println(s"\naccuracy = ${evaluator.evaluate(dfresult)}")

labelCol: label column name (default: label, current: label)
metricName: metric name in evaluation (f1|weightedPrecision|weightedRecall|accuracy) (default: f1, current: f1)
predictionCol: prediction column name (default: prediction, current: prediction)

accuracy = 0.5666666666666667


evaluator = mcEval_ec2e4219e0d6


mcEval_ec2e4219e0d6

**6，保存模型**

In [10]:
model.write.overwrite().save("mymodel.model")

// Now we can optionally save the fitted pipeline to disk
//model.write.overwrite().save("/tmp/spark-logistic-regression-model")

// We can also save this unfit pipeline to disk
//pipeline.write.overwrite().save("/tmp/unfit-lr-model")

In [11]:
//重新载入模型
val model_loaded = PipelineModel.load("mymodel.model")
model_loaded.transform(dftest).select("text","label","prediction").show

+--------------------+-----+----------+
|                text|label|prediction|
+--------------------+-----+----------+
|           spark job|  1.0|       1.0|
|         hello world|  0.0|       1.0|
|           a b c d e|  0.0|       1.0|
|      you can you up|  0.0|       0.0|
|spark is easy to ...|  1.0|       1.0|
+--------------------+-----+----------+



model_loaded = pipeline_3ce68f9bab3a


pipeline_3ce68f9bab3a

### 三，导入数据

可以使用spark.read导入csv，image，libsvm，txt等格式数据。

In [12]:
//导入图片

val dfimage = spark.read.format("image").option("dropInvalid", true).load("data/mllib/images/origin/kittens")
dfimage.printSchema


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



dfimage = [image: struct<origin: string, height: int ... 4 more fields>]


[image: struct<origin: string, height: int ... 4 more fields>]

In [14]:
dfimage.selectExpr("image.*").show

+--------------------+------+-----+---------+----+--------------------+
|              origin|height|width|nChannels|mode|                data|
+--------------------+------+-----+---------+----+--------------------+
|file:///Users/lia...|   311|  300|        3|  16|[C1 C1 C1 C2 C2 C...|
|file:///Users/lia...|   313|  199|        3|  16|[D0 E5 ED CA DF E...|
|file:///Users/lia...|   200|  300|        3|  16|[58 5D 60 58 5D 6...|
|file:///Users/lia...|   296|  300|        3|  16|[CB E6 F4 CA E5 F...|
+--------------------+------+-----+---------+----+--------------------+



### 四，特征工程

spark的特征处理功能主要在 spark.ml.feature 模块中，包括以下一些功能。

* 特征提取：Tf-idf, Word2Vec, CountVectorizer, FeatureHasher

* 特征转换：OneHotEncoderEstimator, Normalizer, Imputer(缺失值填充), StandardScaler, MinMaxScaler, Tokenizer(构建词典), 
  StopWordsRemover, SQLTransformer, Bucketizer, Interaction(交叉项), Binarizer(二值化), n-gram,……

* 特征选择：VectorSlicer(向量切片), RFormula, ChiSqSelector(卡方检验)

* LSH转换：局部敏感哈希广泛用于海量数据中求最邻近，聚类等算法。


**1，CountVectorizer**

CountVectorizer可以提取文本中的词频特征。

In [18]:
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}

val df = spark.createDataFrame(Seq(
  (0, Array("a", "b", "c")),
  (1, Array("a", "b", "b", "c", "a"))
)).toDF("id", "words")

// fit a CountVectorizerModel from the corpus
val cvModel: CountVectorizerModel = new CountVectorizer()
  .setInputCol("words")
  .setOutputCol("features")
  .setVocabSize(3)
  .setMinDF(2)
  .fit(df)

// alternatively, define CountVectorizerModel with a-priori vocabulary
val cvm = new CountVectorizerModel(Array("a", "b", "c"))
  .setInputCol("words")
  .setOutputCol("features")

cvModel.transform(df).show()

cvm.transform(df).show()

+---+---------------+--------------------+
| id|          words|            features|
+---+---------------+--------------------+
|  0|      [a, b, c]|(3,[0,1,2],[1.0,1...|
|  1|[a, b, b, c, a]|(3,[0,1,2],[2.0,2...|
+---+---------------+--------------------+

+---+---------------+--------------------+
| id|          words|            features|
+---+---------------+--------------------+
|  0|      [a, b, c]|(3,[0,1,2],[1.0,1...|
|  1|[a, b, b, c, a]|(3,[0,1,2],[2.0,2...|
+---+---------------+--------------------+



df = [id: int, words: array<string>]
cvModel = cntVec_324b7869593e
cvm = cntVecModel_4947f6142271


cntVecModel_4947f6142271

**2，Tf-IDF**

Tf-IDF可以降低文本频率过高的常用词的权重。

In [15]:
import org.apache.spark.ml.feature.{HashingTF, IDF, Tokenizer}

val sentenceData = spark.createDataFrame(Seq(
  (0.0, "Hi I heard about Spark"),
  (0.0, "I wish Java could use case classes"),
  (1.0, "Logistic regression models are neat")
)).toDF("label", "sentence")

val tokenizer = new Tokenizer().setInputCol("sentence").setOutputCol("words")
val wordsData = tokenizer.transform(sentenceData)

val hashingTF = new HashingTF()
  .setInputCol("words").setOutputCol("rawFeatures").setNumFeatures(20)

val featurizedData = hashingTF.transform(wordsData)
// alternatively, CountVectorizer can also be used to get term frequency vectors

val idf = new IDF().setInputCol("rawFeatures").setOutputCol("features")
val idfModel = idf.fit(featurizedData)   

val rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[0,5,9,17],[0...|
|  0.0|(20,[2,7,9,13,15]...|
|  1.0|(20,[4,6,13,15,18...|
+-----+--------------------+



sentenceData = [label: double, sentence: string]
tokenizer = tok_a56305ad5983
wordsData = [label: double, sentence: string ... 1 more field]
hashingTF = hashingTF_3041530f877f
featurizedData = [label: double, sentence: string ... 2 more fields]
idf = idf_512ac68468f2
idfModel = idf_512ac68468f2
rescaledData = [label: double, sentence: string ... 3 more fields]


[label: double, sentence: string ... 3 more fields]

**3，Word2Vec**

Word2Vec可以使用浅层神经网络提取文本中词的相似语义信息。

In [16]:
import org.apache.spark.ml.feature.Word2Vec
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

// Input data: Each row is a bag of words from a sentence or document.
val documentDF = spark.createDataFrame(Seq(
  "Hi I heard about Spark".split(" "),
  "I wish Java could use case classes".split(" "),
  "Logistic regression models are neat".split(" ")
).map(Tuple1.apply)).toDF("text")

// Learn a mapping from words to Vectors.
val word2Vec = new Word2Vec()
  .setInputCol("text")
  .setOutputCol("result")
  .setVectorSize(3)
  .setMinCount(0)
val model = word2Vec.fit(documentDF)

val result = model.transform(documentDF)
result.collect().foreach { case Row(text: Seq[_], features: Vector) =>
  println(s"Text: [${text.mkString(", ")}] => \nVector: $features\n") }

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.008142343163490296,0.02051363289356232,0.03255096450448036]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.043090314205203734,0.035048123182994974,0.023512658663094044]

Text: [Logistic, regression, models, are, neat] => 
Vector: [0.038572299480438235,-0.03250147425569594,-0.01552378609776497]



documentDF = [text: array<string>]
word2Vec = w2v_9826d4094a99
model = w2v_9826d4094a99
result = [text: array<string>, result: vector]


[text: array<string>, result: vector]

**4， OnHotEncoderEstimator**

OneHotEncoderEstimator可以将类别特征转换成OneHot编码。

In [17]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val df = spark.createDataFrame(Seq(
  (0.0, 1.0),
  (1.0, 0.0),
  (2.0, 1.0),
  (0.0, 2.0),
  (0.0, 1.0),
  (2.0, 0.0)
)).toDF("categoryIndex1", "categoryIndex2")

val encoder = new OneHotEncoderEstimator()
  .setInputCols(Array("categoryIndex1", "categoryIndex2"))
  .setOutputCols(Array("categoryVec1", "categoryVec2"))
val model = encoder.fit(df)

val encoded = model.transform(df)
encoded.show()

+--------------+--------------+-------------+-------------+
|categoryIndex1|categoryIndex2| categoryVec1| categoryVec2|
+--------------+--------------+-------------+-------------+
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           1.0|           0.0|(2,[1],[1.0])|(2,[0],[1.0])|
|           2.0|           1.0|    (2,[],[])|(2,[1],[1.0])|
|           0.0|           2.0|(2,[0],[1.0])|    (2,[],[])|
|           0.0|           1.0|(2,[0],[1.0])|(2,[1],[1.0])|
|           2.0|           0.0|    (2,[],[])|(2,[0],[1.0])|
+--------------+--------------+-------------+-------------+



df = [categoryIndex1: double, categoryIndex2: double]
encoder = oneHotEncoder_53d7e1227a6e
model = oneHotEncoder_53d7e1227a6e
encoded = [categoryIndex1: double, categoryIndex2: double ... 2 more fields]


[categoryIndex1: double, categoryIndex2: double ... 2 more fields]

**5，FeatureHasher**

当特征数量过多时，可以用FeatureHasher来进行降维。

In [18]:
import org.apache.spark.ml.feature.FeatureHasher

val dataset = spark.createDataFrame(Seq(
  (2.2, true, "1", "foo"),
  (3.3, false, "2", "bar"),
  (4.4, false, "3", "baz"),
  (5.5, false, "4", "foo")
)).toDF("real", "bool", "stringNum", "string")

val hasher = new FeatureHasher()
  .setInputCols("real", "bool", "stringNum", "string")
  .setOutputCol("features")

val featurized = hasher.transform(dataset)
featurized.show(false)

+----+-----+---------+------+--------------------------------------------------------+
|real|bool |stringNum|string|features                                                |
+----+-----+---------+------+--------------------------------------------------------+
|2.2 |true |1        |foo   |(262144,[174475,247670,257907,262126],[2.2,1.0,1.0,1.0])|
|3.3 |false|2        |bar   |(262144,[70644,89673,173866,174475],[1.0,1.0,1.0,3.3])  |
|4.4 |false|3        |baz   |(262144,[22406,70644,174475,187923],[1.0,1.0,4.4,1.0])  |
|5.5 |false|4        |foo   |(262144,[70644,101499,174475,257907],[1.0,1.0,5.5,1.0]) |
+----+-----+---------+------+--------------------------------------------------------+



dataset = [real: double, bool: boolean ... 2 more fields]
hasher = featureHasher_c4a5311853be
featurized = [real: double, bool: boolean ... 3 more fields]


[real: double, bool: boolean ... 3 more fields]

**6, IndexToString 和 StringIndexer**

StringIndexer和IndexToString互为逆操作。

前者可以将字符串表示的类别特征转换成数字表示的类别特征。

后者可以将数字表示的类别特征转换成字符串表示的类别特征。

两者一般会同时使用，先用StringIndexer将字符串编码成数字。

经过模型转换后，再调用IndexToString将数字转换成字符串。

StringIndexer是一个Estimator需要被fit。

而IndexToString是一个Transformer可以直接使用。


In [19]:
import org.apache.spark.ml.attribute.Attribute
import org.apache.spark.ml.feature.{IndexToString, StringIndexer}

val df = spark.createDataFrame(Seq(
  (0, "a"),
  (1, "b"),
  (2, "c"),
  (3, "a"),
  (4, "a"),
  (5, "c")
)).toDF("id", "category")

val indexer = new StringIndexer()
  .setInputCol("category")
  .setOutputCol("categoryIndex")
  .fit(df)
val indexed = indexer.transform(df)

println(s"Transformed string column '${indexer.getInputCol}' " +
    s"to indexed column '${indexer.getOutputCol}'")
indexed.show()

val inputColSchema = indexed.schema(indexer.getOutputCol)
println(s"StringIndexer will store labels in output column metadata: " +
    s"${Attribute.fromStructField(inputColSchema).toString}\n")

val converter = new IndexToString()
  .setInputCol("categoryIndex")
  .setOutputCol("originalCategory")

val converted = converter.transform(indexed)

println(s"Transformed indexed column '${converter.getInputCol}' back to original string " +
    s"column '${converter.getOutputCol}' using labels in metadata")
converted.select("id", "categoryIndex", "originalCategory").show()


Transformed string column 'category' to indexed column 'categoryIndex'
+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+

StringIndexer will store labels in output column metadata: {"vals":["a","c","b"],"type":"nominal","name":"categoryIndex"}

Transformed indexed column 'categoryIndex' back to original string column 'originalCategory' using labels in metadata
+---+-------------+----------------+
| id|categoryIndex|originalCategory|
+---+-------------+----------------+
|  0|          0.0|               a|
|  1|          2.0|               b|
|  2|          1.0|               c|
|  3|          0.0|               a|
|  4|          0.0|               a|
|  5|          1.0|               c|
+---+-------------+----------------+



df = [id: int, category: string]
indexer = strIdx_ea624b6a7998
indexed = [id: int, category: string ... 1 more field]
inputColSchema = StructField(categoryIndex,DoubleType,false)
converter = idxToStr_d63f9da12729
converted = [id: int, category: string ... 2 more fields]


[id: int, category: string ... 2 more fields]

In [20]:
indexed.schema(indexer.getOutputCol)

Attribute.fromStructField(inputColSchema).toString

{"vals":["a","c","b"],"type":"nominal","name":"categoryIndex"}

**7，StandardScaler 正态标准化**

In [21]:
import org.apache.spark.ml.feature.StandardScaler

val dataFrame = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

val scaler = new StandardScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")
  .setWithStd(true)
  .setWithMean(false)

// Compute summary statistics by fitting the StandardScaler.
val scalerModel = scaler.fit(dataFrame)

// Normalize each feature to have unit standard deviation.
val scaledData = scalerModel.transform(dataFrame)
scaledData.show()

+-----+--------------------+--------------------+
|label|            features|      scaledFeatures|
+-----+--------------------+--------------------+
|  0.0|(692,[127,128,129...|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|(692,[124,125,126...|


dataFrame = [label: double, features: vector]
scaler = stdScal_9884777b0354
scalerModel = stdScal_9884777b0354
scaledData = [label: double, features: vector ... 1 more field]


[label: double, features: vector ... 1 more field]

**8, MinMax标准化**

In [22]:
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.ml.linalg.Vectors

val df = spark.createDataFrame(Seq(
  (0, Vectors.dense(1.0, 0.1, -1.0)),
  (1, Vectors.dense(2.0, 1.1, 1.0)),
  (2, Vectors.dense(3.0, 10.1, 3.0))
)).toDF("id", "features")

val scaler = new MinMaxScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")

// Compute summary statistics and generate MinMaxScalerModel
val scalerModel = scaler.fit(df)

// rescale each feature to range [min, max].
val scaledData = scalerModel.transform(df)
println(s"Features scaled to range: [${scaler.getMin}, ${scaler.getMax}]")
scaledData.select("features", "scaledFeatures").show()


Features scaled to range: [0.0, 1.0]
+--------------+--------------+
|      features|scaledFeatures|
+--------------+--------------+
|[1.0,0.1,-1.0]| [0.0,0.0,0.0]|
| [2.0,1.1,1.0]| [0.5,0.1,0.5]|
|[3.0,10.1,3.0]| [1.0,1.0,1.0]|
+--------------+--------------+



df = [id: int, features: vector]
scaler = minMaxScal_c35a76973fd4
scalerModel = minMaxScal_c35a76973fd4
scaledData = [id: int, features: vector ... 1 more field]


[id: int, features: vector ... 1 more field]

**9，MaxAbsScaler标准化**

In [23]:
import org.apache.spark.ml.feature.MaxAbsScaler
import org.apache.spark.ml.linalg.Vectors

val df = spark.createDataFrame(Seq(
  (0, Vectors.dense(1.0, 0.1, -8.0)),
  (1, Vectors.dense(2.0, 1.0, -4.0)),
  (2, Vectors.dense(4.0, 10.0, 8.0))
)).toDF("id", "features")

val scaler = new MaxAbsScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")

// Compute summary statistics and generate MaxAbsScalerModel
val scalerModel = scaler.fit(df)

// rescale each feature to range [-1, 1]
val scaledData = scalerModel.transform(df)
scaledData.select("features", "scaledFeatures").show()

+--------------+----------------+
|      features|  scaledFeatures|
+--------------+----------------+
|[1.0,0.1,-8.0]|[0.25,0.01,-1.0]|
|[2.0,1.0,-4.0]|  [0.5,0.1,-0.5]|
|[4.0,10.0,8.0]|   [1.0,1.0,1.0]|
+--------------+----------------+



df = [id: int, features: vector]
scaler = maxAbsScal_8f99413b2cdd
scalerModel = maxAbsScal_8f99413b2cdd
scaledData = [id: int, features: vector ... 1 more field]


[id: int, features: vector ... 1 more field]

**10，SQLTransformer**

可以使用SQL语法将DataFrame进行转换，等效于注册表的作用。

但它可以用于Pipeline中作为Transformer.

In [24]:
import org.apache.spark.ml.feature.SQLTransformer

val df = spark.createDataFrame(
  Seq((0, 1.0, 3.0), (2, 2.0, 5.0))).toDF("id", "v1", "v2")

val sqlTrans = new SQLTransformer().setStatement(
  "SELECT *, (v1 + v2) AS v3, (v1 * v2) AS v4 FROM __THIS__")

sqlTrans.transform(df).show()

+---+---+---+---+----+
| id| v1| v2| v3|  v4|
+---+---+---+---+----+
|  0|1.0|3.0|4.0| 3.0|
|  2|2.0|5.0|7.0|10.0|
+---+---+---+---+----+



df = [id: int, v1: double ... 1 more field]
sqlTrans = sql_f37dc225ff0f


sql_f37dc225ff0f

**11, Imputer**

Imputer转换器可以填充缺失值，缺失值可以用 Double.NaN来表示。

In [25]:
import org.apache.spark.ml.feature.Imputer

val df = spark.createDataFrame(Seq(
  (1.0, Double.NaN),
  (2.0, Double.NaN),
  (Double.NaN, 3.0),
  (4.0, 4.0),
  (5.0, 5.0)
)).toDF("a", "b")

val imputer = new Imputer()
  .setInputCols(Array("a", "b"))
  .setOutputCols(Array("out_a", "out_b"))

val model = imputer.fit(df)
model.transform(df).show()

+---+---+-----+-----+
|  a|  b|out_a|out_b|
+---+---+-----+-----+
|1.0|NaN|  1.0|  4.0|
|2.0|NaN|  2.0|  4.0|
|NaN|3.0|  3.0|  3.0|
|4.0|4.0|  4.0|  4.0|
|5.0|5.0|  5.0|  5.0|
+---+---+-----+-----+



df = [a: double, b: double]
imputer = imputer_4f15731fee62
model = imputer_4f15731fee62


imputer_4f15731fee62

**12 ChiSqSelector**

当label是离散值时，ChiSqSelector选择器可以根据Chi2检验统计量筛选特征。

In [26]:
import org.apache.spark.ml.feature.ChiSqSelector
import org.apache.spark.ml.linalg.Vectors

val data = Seq(
  (7, Vectors.dense(0.0, 0.0, 18.0, 1.0), 1.0),
  (8, Vectors.dense(0.0, 1.0, 12.0, 0.0), 0.0),
  (9, Vectors.dense(1.0, 0.0, 15.0, 0.1), 0.0)
)

val df = spark.createDataset(data).toDF("id", "features", "clicked")

val selector = new ChiSqSelector()
  .setNumTopFeatures(1)
  .setFeaturesCol("features")
  .setLabelCol("clicked")
  .setOutputCol("selectedFeatures")

val result = selector.fit(df).transform(df)

println(s"ChiSqSelector output with top ${selector.getNumTopFeatures} features selected")
result.show()

ChiSqSelector output with top 1 features selected
+---+------------------+-------+----------------+
| id|          features|clicked|selectedFeatures|
+---+------------------+-------+----------------+
|  7|[0.0,0.0,18.0,1.0]|    1.0|          [18.0]|
|  8|[0.0,1.0,12.0,0.0]|    0.0|          [12.0]|
|  9|[1.0,0.0,15.0,0.1]|    0.0|          [15.0]|
+---+------------------+-------+----------------+



data = List((7,[0.0,0.0,18.0,1.0],1.0), (8,[0.0,1.0,12.0,0.0],0.0), (9,[1.0,0.0,15.0,0.1],0.0))
df = [id: int, features: vector ... 1 more field]
selector = chiSqSelector_78fe2d0ce95d
result = [id: int, features: vector ... 2 more fields]


[id: int, features: vector ... 2 more fields]

**13，局部敏感哈希（LSH）**

Locality Sensitive Hashing 是一种广泛使用的哈希技巧，常用于求海量数据的最邻近，聚类和离群值检测等任务中。

局部敏感哈希的特性是将距离较近的点以很大的概率通过哈希作用映射到相同的桶内，而将距离较远的点以很大的概率通过哈希作用映射到不同的桶中。

BucketedRandomProjectionLSH 使用Euclidean distance作为距离度量，取点在某个随机方向的投影结果作为哈希值。

MinHashLSH 使用Jaccard distance作为距离度量，它是衡量两个集合差异性的一个指标，MinHashLSH取集合中所有元素的哈希值的最小值作为哈希值。

杰卡德距离计算公式如下:$$d(A,B)=1−\frac{|A∩B|}{|A∪B|}$$


In [27]:
import org.apache.spark.ml.feature.BucketedRandomProjectionLSH
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.col

val dfA = spark.createDataFrame(Seq(
  (0, Vectors.dense(1.0, 1.0)),
  (1, Vectors.dense(1.0, -1.0)),
  (2, Vectors.dense(-1.0, -1.0)),
  (3, Vectors.dense(-1.0, 1.0))
)).toDF("id", "features")

val dfB = spark.createDataFrame(Seq(
  (4, Vectors.dense(1.0, 0.0)),
  (5, Vectors.dense(-1.0, 0.0)),
  (6, Vectors.dense(0.0, 1.0)),
  (7, Vectors.dense(0.0, -1.0))
)).toDF("id", "features")

val key = Vectors.dense(1.0, 0.0)

val brp = new BucketedRandomProjectionLSH()
  .setBucketLength(2.0)
  .setNumHashTables(3)
  .setInputCol("features")
  .setOutputCol("hashes")

val model = brp.fit(dfA)

// Feature Transformation
println("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(dfA).show()

// Compute the locality sensitive hashes for the input rows, then perform approximate
// similarity join.
// We could avoid computing hashes by passing in the already-transformed dataset, e.g.
// `model.approxSimilarityJoin(transformedA, transformedB, 1.5)`
println("Approximately joining dfA and dfB on Euclidean distance smaller than 1.5:")
model.approxSimilarityJoin(dfA, dfB, 1.5, "EuclideanDistance")
  .select(col("datasetA.id").alias("idA"),
    col("datasetB.id").alias("idB"),
    col("EuclideanDistance")).show()

// Compute the locality sensitive hashes for the input rows, then perform approximate nearest
// neighbor search.
// We could avoid computing hashes by passing in the already-transformed dataset, e.g.
// `model.approxNearestNeighbors(transformedA, key, 2)`
println("Approximately searching dfA for 2 nearest neighbors of the key:")
model.approxNearestNeighbors(dfA, key, 2).show()


The hashed dataset where hashed values are stored in the column 'hashes':
+---+-----------+--------------------+
| id|   features|              hashes|
+---+-----------+--------------------+
|  0|  [1.0,1.0]|[[0.0], [0.0], [-...|
|  1| [1.0,-1.0]|[[-1.0], [-1.0], ...|
|  2|[-1.0,-1.0]|[[-1.0], [-1.0], ...|
|  3| [-1.0,1.0]|[[0.0], [0.0], [-...|
+---+-----------+--------------------+

Approximately joining dfA and dfB on Euclidean distance smaller than 1.5:
+---+---+-----------------+
|idA|idB|EuclideanDistance|
+---+---+-----------------+
|  1|  4|              1.0|
|  0|  6|              1.0|
|  1|  7|              1.0|
|  3|  5|              1.0|
|  0|  4|              1.0|
|  3|  6|              1.0|
|  2|  7|              1.0|
|  2|  5|              1.0|
+---+---+-----------------+

Approximately searching dfA for 2 nearest neighbors of the key:
+---+----------+--------------------+-------+
| id|  features|              hashes|distCol|
+---+----------+--------------------+-------+


dfA = [id: int, features: vector]
dfB = [id: int, features: vector]
key = [1.0,0.0]
brp = brp-lsh_9d4f769e11c9
model = brp-lsh_9d4f769e11c9


brp-lsh_9d4f769e11c9

In [28]:
import org.apache.spark.ml.feature.MinHashLSH
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.col

val dfA = spark.createDataFrame(Seq(
  (0, Vectors.sparse(6, Seq((0, 1.0), (1, 1.0), (2, 1.0)))),
  (1, Vectors.sparse(6, Seq((2, 1.0), (3, 1.0), (4, 1.0)))),
  (2, Vectors.sparse(6, Seq((0, 1.0), (2, 1.0), (4, 1.0))))
)).toDF("id", "features")

val dfB = spark.createDataFrame(Seq(
  (3, Vectors.sparse(6, Seq((1, 1.0), (3, 1.0), (5, 1.0)))),
  (4, Vectors.sparse(6, Seq((2, 1.0), (3, 1.0), (5, 1.0)))),
  (5, Vectors.sparse(6, Seq((1, 1.0), (2, 1.0), (4, 1.0))))
)).toDF("id", "features")

val key = Vectors.sparse(6, Seq((1, 1.0), (3, 1.0)))

val mh = new MinHashLSH()
  .setNumHashTables(5)
  .setInputCol("features")
  .setOutputCol("hashes")

val model = mh.fit(dfA)

// Feature Transformation
println("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(dfA).show()

// Compute the locality sensitive hashes for the input rows, then perform approximate
// similarity join.
// We could avoid computing hashes by passing in the already-transformed dataset, e.g.
// `model.approxSimilarityJoin(transformedA, transformedB, 0.6)`
println("Approximately joining dfA and dfB on Jaccard distance smaller than 0.6:")
model.approxSimilarityJoin(dfA, dfB, 0.6, "JaccardDistance")
  .select(col("datasetA.id").alias("idA"),
    col("datasetB.id").alias("idB"),
    col("JaccardDistance")).show()

// Compute the locality sensitive hashes for the input rows, then perform approximate nearest
// neighbor search.
// We could avoid computing hashes by passing in the already-transformed dataset, e.g.
// `model.approxNearestNeighbors(transformedA, key, 2)`
// It may return less than 2 rows when not enough approximate near-neighbor candidates are
// found.
println("Approximately searching dfA for 2 nearest neighbors of the key:")
model.approxNearestNeighbors(dfA, key, 2).show()


The hashed dataset where hashed values are stored in the column 'hashes':
+---+--------------------+--------------------+
| id|            features|              hashes|
+---+--------------------+--------------------+
|  0|(6,[0,1,2],[1.0,1...|[[2.25592966E8], ...|
|  1|(6,[2,3,4],[1.0,1...|[[2.25592966E8], ...|
|  2|(6,[0,2,4],[1.0,1...|[[2.25592966E8], ...|
+---+--------------------+--------------------+

Approximately joining dfA and dfB on Jaccard distance smaller than 0.6:
+---+---+---------------+
|idA|idB|JaccardDistance|
+---+---+---------------+
|  1|  4|            0.5|
|  0|  5|            0.5|
|  1|  5|            0.5|
|  2|  5|            0.5|
+---+---+---------------+

Approximately searching dfA for 2 nearest neighbors of the key:
+---+--------------------+--------------------+-------+
| id|            features|              hashes|distCol|
+---+--------------------+--------------------+-------+
|  1|(6,[2,3,4],[1.0,1...|[[2.25592966E8], ...|   0.75|
+---+---------------

dfA = [id: int, features: vector]
dfB = [id: int, features: vector]
key = (6,[1,3],[1.0,1.0])
mh = mh-lsh_dee13c9ca2f9
model = mh-lsh_dee13c9ca2f9


mh-lsh_dee13c9ca2f9

### 五，分类模型

mllib支持常见的机器学习分类模型：逻辑回归，SoftMax回归，决策树，随机森林，梯度提升树，线性支持向量机，朴素贝叶斯，One-Vs-Rest，以及多层感知机模型。这些模型的接口使用方法基本大同小异，下面仅仅列举常用的决策树，随机森林和梯度提升树的使用作为示范。更多范例参见官方文档。

**1，决策树**

In [12]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.DecisionTreeClassificationModel
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

// Load the data stored in LIBSVM format as a DataFrame.
val data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(data)
// Automatically identify categorical features, and index them.
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4) // features with > 4 distinct values are treated as continuous.
  .fit(data)

// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

// Train a DecisionTree model.
val tree = new DecisionTreeClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")

// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Chain indexers and tree in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, featureIndexer, tree, labelConverter))

// Train model. This also runs the indexers.
val model = pipeline.fit(trainingData)

// Make predictions.
val predictions = model.transform(testData)

// Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println(s"Test Error = ${(1.0 - accuracy)}")

val treeModel = model.stages(2).asInstanceOf[DecisionTreeClassificationModel]
println(s"Learned classification tree model:\n ${treeModel.toDebugString}")

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[100,101,102...|
|           0.0|  0.0|(692,[122,123,148...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[126,127,128...|
|           0.0|  0.0|(692,[126,127,128...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0.05714285714285716
Learned classification tree model:
 DecisionTreeClassificationModel (uid=dtc_af858bdfe537) of depth 1 with 3 nodes
  If (feature 406 <= 12.0)
   Predict: 1.0
  Else (feature 406 > 12.0)
   Predict: 0.0



data = [label: double, features: vector]
labelIndexer = strIdx_85fcf8c3bc07
featureIndexer = vecIdx_b22be23dd44b
trainingData = [label: double, features: vector]
testData = [label: double, features: vector]


tree: org.apache.spark...


[label: double, features: vector]

**2，随机森林**

In [14]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

// Load and parse the data file, converting it to a DataFrame.
val data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(data)
// Automatically identify categorical features, and index them.
// Set maxCategories so features with > 4 distinct values are treated as continuous.
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)
  .fit(data)

// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

// Train a RandomForest model.
val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(10)

// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Chain indexers and forest in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, featureIndexer, rf, labelConverter))

// Train model. This also runs the indexers.
val model = pipeline.fit(trainingData)

// Make predictions.
val predictions = model.transform(testData)

// Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println(s"Test Error = ${(1.0 - accuracy)}")

val rfModel = model.stages(2).asInstanceOf[RandomForestClassificationModel]
println(s"Learned classification forest model:\n ${rfModel.toDebugString}")

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[98,99,100,1...|
|           0.0|  0.0|(692,[122,123,124...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[124,125,126...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0.0
Learned classification forest model:
 RandomForestClassificationModel (uid=rfc_00687b80a80b) with 10 trees
  Tree 0 (weight 1.0):
    If (feature 412 <= 8.0)
     If (feature 323 <= 7.5)
      If (feature 630 <= 5.0)
       Predict: 0.0
      Else (feature 630 > 5.0)
       Predict: 1.0
     Else (feature 323 > 7.5)
      Predict: 0.0
    Else (feature 412 > 8.0)
     Predict: 1.0
  Tree 1 (weight 1.0):
    If (feature 463 <= 2.0)
     If (feature 605 <= 12.0)
      If (feature 207 <= 2.0)
       Predict: 0.0
      Else (feature 207 > 2.0)
       Predict: 

data = [label: double, features: vector]
labelIndexer = strIdx_489d7a427841
featureIndexer = vecIdx_b573479bab67
trainingData = [label: double, features: vector]
testData = [label: double, features: vector]


rf: org.apache.spark.ml.classification.RandomForestClassifier...


[label: double, features: vector]

**3，梯度提升树**

In [15]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

// Load and parse the data file, converting it to a DataFrame.
val data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val labelIndexer = new StringIndexer()
  .setInputCol("label")
  .setOutputCol("indexedLabel")
  .fit(data)
// Automatically identify categorical features, and index them.
// Set maxCategories so features with > 4 distinct values are treated as continuous.
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)
  .fit(data)

// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

// Train a GBT model.
val gbt = new GBTClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setMaxIter(10)
  .setFeatureSubsetStrategy("auto")

// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

// Chain indexers and GBT in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, featureIndexer, gbt, labelConverter))

// Train model. This also runs the indexers.
val model = pipeline.fit(trainingData)

// Make predictions.
val predictions = model.transform(testData)

// Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println(s"Test Error = ${1.0 - accuracy}")

val gbtModel = model.stages(2).asInstanceOf[GBTClassificationModel]
println(s"Learned classification GBT model:\n ${gbtModel.toDebugString}")

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[123,124,125...|
|           0.0|  0.0|(692,[123,124,125...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[126,127,128...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0.0
Learned classification GBT model:
 GBTClassificationModel (uid=gbtc_3c7196e716c1) with 10 trees
  Tree 0 (weight 1.0):
    If (feature 434 <= 70.5)
     If (feature 99 in {2.0})
      Predict: -1.0
     Else (feature 99 not in {2.0})
      Predict: 1.0
    Else (feature 434 > 70.5)
     Predict: -1.0
  Tree 1 (weight 0.1):
    If (feature 490 <= 43.0)
     If (feature 344 <= 253.5)
      Predict: 0.4768116880884702
     Else (feature 344 > 253.5)
      Predict: -0.4768116880884694
    Else (feature 490 > 43.0)
     Predict: -0.47681168808846996
  Tree 2 (w

data = [label: double, features: vector]
labelIndexer = strIdx_5aa2ccbbcbe3
featureIndexer = vecIdx_084188469920
trainingData = [label: double, features: vector]
testData = [label: double, features: vector]
gbt = gbtc_3c7196e716c1


label...


gbtc_3c7196e716c1

In [18]:
data.show

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



### 六，回归模型

mllib支持常见的回归模型，如线性回归，广义线性回归，决策树回归，随机森林回归，梯度提升树回归，生存回归，保序回归。

**1，线性回归**

In [30]:
import org.apache.spark.ml.regression.LinearRegression

// Load training data
val training = spark.read.format("libsvm")
  .load("data/mllib/sample_linear_regression_data.txt")

val linear = new LinearRegression()
  .setMaxIter(10)
  .setRegParam(0.3)
  .setElasticNetParam(0.8)

// Fit the model
val model = linear.fit(training)

// Print the coefficients and intercept for linear regression
println(s"Coefficients: ${model.coefficients} Intercept: ${model.intercept}")

// Summarize the model over the training set and print out some metrics
val trainingSummary = model.summary
println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"objectiveHistory: [${trainingSummary.objectiveHistory.mkString(",")}]")
trainingSummary.residuals.show()
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"r2: ${trainingSummary.r2}")

Coefficients: [0.0,0.32292516677405936,-0.3438548034562218,1.9156017023458414,0.05288058680386263,0.765962720459771,0.0,-0.15105392669186682,-0.21587930360904642,0.22025369188813426] Intercept: 0.1598936844239736
numIterations: 7
objectiveHistory: [0.49999999999999994,0.4967620357443381,0.4936361664340463,0.4936351537897608,0.4936351214177871,0.49363512062528014,0.4936351206216114]
+--------------------+
|           residuals|
+--------------------+
|  -9.889232683103197|
|  0.5533794340053554|
|  -5.204019455758823|
| -20.566686715507508|
|    -9.4497405180564|
|  -6.909112502719486|
|  -10.00431602969873|
|   2.062397807050484|
|  3.1117508432954772|
| -15.893608229419382|
|  -5.036284254673026|
|   6.483215876994333|
|  12.429497299109002|
|  -20.32003219007654|
| -2.0049838218725005|
| -17.867901734183793|
|   7.646455887420495|
| -2.2653482182417406|
|-0.10308920436195645|
|  -1.380034070385301|
+--------------------+
only showing top 20 rows

RMSE: 10.189077167598475
r2: 0.022861

training = [label: double, features: vector]
linear = linReg_6f85a0c3fe39
model = linReg_6f85a0c3fe39
trainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@75347195


org.apache.spark.ml.regression.LinearRegressionTrainingSummary@75347195

**2，决策树回归**

In [33]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.feature.VectorIndexer
import org.apache.spark.ml.regression.DecisionTreeRegressionModel
import org.apache.spark.ml.regression.DecisionTreeRegressor

// Load the data stored in LIBSVM format as a DataFrame.
val data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

// Automatically identify categorical features, and index them.
// Here, we treat features with > 4 distinct values as continuous.
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)
  .fit(data)

// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, testData) = data.randomSplit(Array(0.7, 0.3))

// Train a DecisionTree model.
val tree = new DecisionTreeRegressor()
  .setLabelCol("label")
  .setFeaturesCol("indexedFeatures")

// Chain indexer and tree in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(featureIndexer, tree))

// Train model. This also runs the indexer.
val model = pipeline.fit(trainingData)

// Make predictions.
val predictions = model.transform(testData)

// Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

val treeModel = model.stages(1).asInstanceOf[DecisionTreeRegressionModel]
println(s"Learned regression tree model:\n ${treeModel.toDebugString}")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       1.0|  0.0|(692,[98,99,100,1...|
|       0.0|  0.0|(692,[122,123,124...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
|       0.0|  0.0|(692,[124,125,126...|
+----------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.1796053020267749
Learned regression tree model:
 DecisionTreeRegressionModel (uid=dtr_8d4d7bdd47fe) of depth 2 with 5 nodes
  If (feature 434 <= 79.5)
   If (feature 99 in {0.0})
    Predict: 0.0
   Else (feature 99 not in {0.0})
    Predict: 1.0
  Else (feature 434 > 79.5)
   Predict: 1.0



data = [label: double, features: vector]
featureIndexer = vecIdx_f610d0ae6ae1
trainingData = [label: double, features: vector]
testData = [label: double, features: vector]
tree = dtr_8d4d7bdd47fe
pipeline = pipeline_4a9d0a244a26


model: org.apache.spark...


pipeline_4a9d0a244a26

### 七，聚类模型

mllib支持的聚类模型较少，主要有K均值聚类，高斯混合模型GMM，以及二分的K均值，隐含狄利克雷分布LDA模型等。

通过调用其它库，也可以基于spark实现分布式的DBSCAN等聚类算法。

**1，K均值聚类**

In [36]:
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.evaluation.ClusteringEvaluator

// Loads data.
val dfdata = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

// Trains a k-means model.
val kmeans = new KMeans().setK(2).setSeed(1L)
val model = kmeans.fit(dfdata)

// Make predictions
val dfpredict = model.transform(dfdata)

// Evaluate clustering by computing Silhouette score
val evaluator = new ClusteringEvaluator()

val silhouette = evaluator.evaluate(dfpredict)
println(s"Silhouette with squared euclidean distance = $silhouette")

// Shows the result.
println("Cluster Centers: ")
model.clusterCenters.foreach(println)

Silhouette with squared euclidean distance = 0.9997530305375207
Cluster Centers: 
[0.1,0.1,0.1]
[9.1,9.1,9.1]


dfdata = [label: double, features: vector]
kmeans = kmeans_d45e6549d6f9
model = kmeans_d45e6549d6f9
dfpredict = [label: double, features: vector ... 1 more field]
evaluator = cluEval_91c4b9890b93
silhouette = 0.9997530305375207


0.9997530305375207

**2，高斯混合模型**

In [37]:
import org.apache.spark.ml.clustering.GaussianMixture

// Loads data
val dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

// Trains Gaussian Mixture Model
val gmm = new GaussianMixture()
  .setK(2)
val model = gmm.fit(dataset)

// output parameters of mixture model model
for (i <- 0 until model.getK) {
  println(s"Gaussian $i:\nweight=${model.weights(i)}\n" +
      s"mu=${model.gaussians(i).mean}\nsigma=\n${model.gaussians(i).cov}\n")
}

Gaussian 0:
weight=0.5
mu=[0.10000000000001552,0.10000000000001552,0.10000000000001552]
sigma=
0.006666666666806454  0.006666666666806454  0.006666666666806454  
0.006666666666806454  0.006666666666806454  0.006666666666806454  
0.006666666666806454  0.006666666666806454  0.006666666666806454  

Gaussian 1:
weight=0.5
mu=[9.099999999999984,9.099999999999984,9.099999999999984]
sigma=
0.006666666666812185  0.006666666666812185  0.006666666666812185  
0.006666666666812185  0.006666666666812185  0.006666666666812185  
0.006666666666812185  0.006666666666812185  0.006666666666812185  



dataset = [label: double, features: vector]
gmm = GaussianMixture_93ea01f101df
model = GaussianMixture_93ea01f101df


GaussianMixture_93ea01f101df

**3, 二分K均值 Bisecting k-means**

Bisecting k-means是一种自上而下的层次聚类算法。所有的样本点开始时属于一个cluster,然后不断通过K均值二分裂得到多个cluster。

In [38]:
import org.apache.spark.ml.clustering.BisectingKMeans

// Loads data.
val dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")

// Trains a bisecting k-means model.
val bkm = new BisectingKMeans().setK(2).setSeed(1)
val model = bkm.fit(dataset)

// Evaluate clustering.
val cost = model.computeCost(dataset)
println(s"Within Set Sum of Squared Errors = $cost")

// Shows the result.
println("Cluster Centers: ")
val centers = model.clusterCenters
centers.foreach(println)


Within Set Sum of Squared Errors = 0.11999999999994547
Cluster Centers: 
[0.1,0.1,0.1]
[9.1,9.1,9.1]


dataset = [label: double, features: vector]
bkm = bisecting-kmeans_596ee31355ba
model = bisecting-kmeans_596ee31355ba
cost = 0.11999999999994547
centers = Array([0.1,0.1,0.1], [9.1,9.1,9.1])


Array([0.1,0.1,0.1], [9.1,9.1,9.1])

### 八，降维模型

### 九，模型优化

### 十，统计工具